In [61]:
from glob import glob
import rosbag
import rospy
import ros_numpy
import bagpy
import cv2
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import argparse
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError
from PIL import Image
import open3d
from ros_tools import rostools


def initialization():
    global file, depth_path

    output_dir = bag_file[:-4]

    valid = rt.verify_cv_bridge()
    # verify_ROS_connection()

    depth_path = output_dir +'/depth_pcd/'

    if not os.path.exists(depth_path):
        os.makedirs(depth_path)
        print("Create Folder at Designated Address depth_path...")



def bag2csv(bag_file):
    b = bagpy.bagreader(bag_file)
    b.message_by_topic('/sync_pose_pan')
    b.message_by_topic('/sync_pose_camhand')
    print("Complete Saving csv.")


# def bag2pcd(bag_file, pcd_topic):
#     bridge = CvBridge()
#     start = time.time()
    
#     with rosbag.Bag(bag_file, 'r') as bag:
#         for topic, depth_msg, t in bag.read_messages(pcd_topic):
#             pcd = rt.rospc_to_o3dpc(depth_msg)
#             save_dir = os.path.join(depth_path, f'{t}.ply')
#             open3d.io.write_point_cloud(save_dir, pcd)
#             end = time.time()
#             print(end - start)
#             print("Complete Saving")

def bag2Pointcloud(bag_file, pcd_topic):
    conversion = 0.179
    with rosbag.Bag(bag_file, 'r') as bag:
        count = 0
        start = time.time()
        # for topic, depth_msg, t in bag.read_messages(args.ambf_pcd_topic):
        #     ambf_pcd = rt.rospc_to_o3dpc(depth_msg)
            
        #     print(path[0] +' '+ str(count))
        #     ambf_file_name = os.path.join(path[0], str(count)) + '.ply'
        #     o3d.io.write_point_cloud(ambf_file_name, ambf_pcd)
        #     count += 1
        #     if count == 1:
        #         break
        

        count = 0
        for topic, depth_msg, t in bag.read_messages(pcd_topic):
            pcd = rt.rospc_to_o3dpc(depth_msg)
            
            R_ros2cv = np.array([[0,1,0],[0,0,-1],[-1,0,0]])  
            # R_ros2cv = np.identity(3)        
            pcd.points = o3d.utility.Vector3dVector((R_ros2cv@(np.asarray(pcd.points)*conversion).transpose(-1,-2)).transpose(-1,-2))
            save_dir = os.path.join(depth_path, str(count)) + '.ply'
            o3d.io.write_point_cloud(save_dir, pcd)
            count += 1
            
        end = time.time()
        print(end - start)
        print("Complete Saving")

def main():
    bag2csv(bag_file)
    bag2Pointcloud(bag_file, pcd_topic)


if __name__ == '__main__':
    rt = rostools()
    bridge = CvBridge()
    global args, bag_file
    bag_file = '/media/shc/Elements/Twin-S_data/post_sync/post_sync.bag'
    pcd_topic = '/sync_pcd/DepthData'
    initialization()
    main()

Create Folder at Designated Address depth_path...
[INFO]  Data folder /media/shc/Elements/Twin-S_data/post_sync/post_sync already exists. Not creating.
Complete Saving csv.
11.323632717132568
Complete Saving


In [55]:
import open3d as o3d
pcd_source = o3d.io.read_point_cloud('/media/shc/Elements/Twin-S_data/post_sync/post_sync/depth_pcd/0.ply')
o3d.visualization.draw_geometries_with_editing([pcd_source])

In [57]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
            size=0.1, origin=[0, 0, 0])
pcd_source = o3d.io.read_point_cloud('/media/shc/Elements/Twin-S_data/post_sync/post_sync/cropped_1.ply')

# pcd_target = o3d.io.read_point_cloud('/media/shc/Elements/Twin-S_data/post_sync/post_sync/depth_pcd/0.ply')
# o3d.visualization.draw_geometries_with_editing([pcd_target])
o3d.visualization.draw_geometries([pcd_source, mesh_frame])

In [62]:
import sys
sys.path.insert(0, '../optical_tracking')
from phantom_registration import manual_registration, draw_registration_result

def phantom_registration(pcd_source, T_c_p_path=None):

    pcd_target = o3d.io.read_point_cloud("../data/phantom_point-cloud_data/phacon_exp_3.ply")
    pcd_target = pcd_target.voxel_down_sample(voxel_size=0.001)

    # manual registration to initialize
    init_result = manual_registration(pcd_source, pcd_target)

    threshold = 0.006
    loss = o3d.pipelines.registration.TukeyLoss(k=0.005)

    trans_init_icp = init_result.transformation

    result = o3d.pipelines.registration.registration_icp(
        pcd_source, pcd_target, threshold, trans_init_icp,
        o3d.pipelines.registration.TransformationEstimationPointToPlane(loss))
        
    draw_registration_result(pcd_source, pcd_target, result.transformation)
    print(result)
    print(result.transformation)


pcd_source = o3d.io.read_point_cloud('/media/shc/Elements/Twin-S_data/post_sync/post_sync/cropped_1.ply')
phantom_registration(pcd_source)

Visualization of two point clouds before manual alignment

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #4288 (0.16, -0.11, 0.45) to add in queue.
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #2654 (0.17, -0.086, 0.43) to add in queue.
[Open3D INFO] Picked point #3888 (0.19, -0.1, 0.44) to add in queue.


1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #20803 (-0.019, 0.0032, 0.0049) to add in queue.
[Open3D INFO] Picked point #26320 (0.0051, 0.0074, 0.024) to add in queue.
[Open3D INFO] Picked point #6232 (0.0044, 0.013, 0.00068) to add in queue.

Compute a rough tran

In [65]:
import pandas as pd
from scipy.spatial.transform import Rotation as R
sys.path.insert(0, '../util')
from Solver import solver
sol = solver()

def getRealPose(idx, pd_data):
    df = pd_data
    pose_x = df['pose.position.x'][idx]*1000
    pose_y = df['pose.position.y'][idx]*1000
    pose_z = df['pose.position.z'][idx]*1000
    orin_x = df['pose.orientation.x'][idx]
    orin_y = df['pose.orientation.y'][idx]
    orin_z = df['pose.orientation.z'][idx]
    orin_w = df['pose.orientation.w'][idx]
    pose = np.array([pose_x, pose_y, pose_z, orin_x, orin_y, orin_z, orin_w])
    # print(real_pose, '\n', pose)
    return pose

In [98]:

opti_dir = f'/media/shc/Elements/Twin-S_data/post_sync/post_sync'
params_save_dir = opti_dir
T_p_pb = np.load('../params/phacon2pan_4.npy')
T_cb_c =  np.load('../params/hand_eye_X_optimized_0321.npy')
T_pb_p = sol.invTransformation(T_p_pb)

pose_pan_data = os.path.join(opti_dir, 'sync_pose_pan.csv')
pose_cam_data = os.path.join(opti_dir, 'sync_pose_camhand.csv')

df_pan = pd.read_csv(pose_pan_data)
df_cam_hand = pd.read_csv(pose_cam_data)

# assert len(df_pan == df_cam_hand)
num_frames = len(df_pan)

T_o_p_batch = np.zeros([num_frames, 4, 4])
T_o_cb_batch = np.zeros([num_frames, 4, 4])
for i in range(num_frames):
    quaternion_pb = getRealPose(i, df_pan)
    quaternion_cb = getRealPose(i, df_cam_hand)
    _, T_o_pb = sol.seven2trans(quaternion_pb)
    _, T_o_cb = sol.seven2trans(quaternion_cb)
    # T_o_p_batch[i, :, :] = T_o_pb @ T_pb_p
    # T_o_cb_batch[i,:,:] = T_o_cb
    T_o_p = T_o_pb @ T_pb_p
    T_c_p_real = sol.invTransformation(T_cb_c)@sol.invTransformation(T_o_cb)@T_o_p
    
    print(np.linalg.norm(T_o_p[:3, 3]))
    break
# np.save(os.path.join(params_save_dir, f'T_o_p_opti{opti_num}_0321.npy'), T_o_p_batch)
# np.save(os.path.join(params_save_dir, f'T_o_cb_opti{opti_num}_0321.npy'), T_o_cb_batch)


2130.533141002058
